In [52]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np


In [53]:
link = "https://de.wikipedia.org/wiki/NUTS:DE"

table_id = pd.read_html(link,header=0)[0]



table_id.head(5)

,NUTS 1,NUTS 1.1,NUTS 2,NUTS 2.1,NUTS 3,NUTS 3.1
0,Code,Name,Code,Name,Code,Name
1,DE1,Baden-Württemberg,DE11,Stuttgart,DE111,"Stuttgart, Stadtkreis"
2,DE1,Baden-Württemberg,DE11,Stuttgart,DE112,Böblingen
3,DE1,Baden-Württemberg,DE11,Stuttgart,DE113,Esslingen
4,DE1,Baden-Württemberg,DE11,Stuttgart,DE114,Göppingen


In [54]:
table_id.rename(columns={'NUTS 3':'Code',
                        'NUTS 3.1':'Name'}, inplace=True)

table_id=table_id.drop(index=0).sort_values(by='Name')
table_id=table_id[['NUTS 1.1','NUTS 2.1', 'Code', 'Name']]
table_id

,NUTS 1.1,NUTS 2.1,Code,Name
296,Rheinland-Pfalz,Koblenz,DEB12,Ahrweiler
131,Bayern,Schwaben,DE275,Aichach-Friedberg
40,Baden-Württemberg,Tübingen,DE145,Alb-Donau-Kreis
399,Thüringen,Thüringen,DEG0M,Altenburger Land
297,Rheinland-Pfalz,Koblenz,DEB13,Altenkirchen (Westerwald)
...,...,...,...,...
117,Bayern,Unterfranken,DE263,"Würzburg, Kreisfreie Stadt"
126,Bayern,Unterfranken,DE26C,"Würzburg, Landkreis"
38,Baden-Württemberg,Tübingen,DE143,Zollernalbkreis
320,Rheinland-Pfalz,Rheinhessen-Pfalz,DEB3A,"Zweibrücken, Kreisfreie Stadt"


In [55]:
link_lk = "https://de.wikipedia.org/wiki/Liste_der_Landkreise_in_Deutschland"

table_lk = pd.read_html(link_lk,header=0)[0].sort_values(by='Landkreis/Kreis')

table_lk=table_lk[['Landkreis/Kreis', 'UZ']]
table_lk.rename(columns={'Landkreis/Kreis':'Name'}, inplace=True)


table_lk.head(5)

,Name,UZ
0,"Aachen, Städteregion[FN 1]","AC, MON"
1,Ahrweiler,AW
2,Aichach-Friedberg,"AIC, FDB"
3,Alb-Donau-Kreis,UL
4,Altenburger Land,"ABG, SLN"


In [56]:
table_lk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 294 entries, 0 to 293
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    294 non-null    object
 1   UZ      294 non-null    object
dtypes: object(2)
memory usage: 6.9+ KB


In [57]:
table_id.info()

<class 'pandas.core.frame.DataFrame'>
Index: 402 entries, 296 to 346
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NUTS 1.1  402 non-null    object
 1   NUTS 2.1  402 non-null    object
 2   Code      402 non-null    object
 3   Name      402 non-null    object
dtypes: object(4)
memory usage: 15.7+ KB


In [58]:
df = pd.merge(table_id, table_lk, how="left", on="Name")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NUTS 1.1  402 non-null    object
 1   NUTS 2.1  402 non-null    object
 2   Code      402 non-null    object
 3   Name      402 non-null    object
 4   UZ        260 non-null    object
dtypes: object(5)
memory usage: 15.8+ KB


In [59]:
df

,NUTS 1.1,NUTS 2.1,Code,Name,UZ
0,Rheinland-Pfalz,Koblenz,DEB12,Ahrweiler,AW
1,Bayern,Schwaben,DE275,Aichach-Friedberg,"AIC, FDB"
2,Baden-Württemberg,Tübingen,DE145,Alb-Donau-Kreis,UL
3,Thüringen,Thüringen,DEG0M,Altenburger Land,"ABG, SLN"
4,Rheinland-Pfalz,Koblenz,DEB13,Altenkirchen (Westerwald),AK
...,...,...,...,...,...
397,Bayern,Unterfranken,DE263,"Würzburg, Kreisfreie Stadt",NaN
398,Bayern,Unterfranken,DE26C,"Würzburg, Landkreis",NaN
399,Baden-Württemberg,Tübingen,DE143,Zollernalbkreis,"BL, HCH"
400,Rheinland-Pfalz,Rheinhessen-Pfalz,DEB3A,"Zweibrücken, Kreisfreie Stadt",NaN


In [60]:
df_nan=df[df['UZ'].isnull()]
df_nan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, 8 to 400
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NUTS 1.1  142 non-null    object
 1   NUTS 2.1  142 non-null    object
 2   Code      142 non-null    object
 3   Name      142 non-null    object
 4   UZ        0 non-null      object
dtypes: object(5)
memory usage: 6.7+ KB


In [61]:
df_nan.head(20)

,NUTS 1.1,NUTS 2.1,Code,Name,UZ
8,Bayern,Oberpfalz,DE231,"Amberg, Kreisfreie Stadt",NaN
12,Bayern,Mittelfranken,DE251,"Ansbach, Kreisfreie Stadt",NaN
13,Bayern,Mittelfranken,DE256,"Ansbach, Landkreis",NaN
14,Bayern,Unterfranken,DE261,"Aschaffenburg, Kreisfreie Stadt",NaN
15,Bayern,Unterfranken,DE264,"Aschaffenburg, Landkreis",NaN
16,Bayern,Schwaben,DE271,"Augsburg, Kreisfreie Stadt",NaN
17,Bayern,Schwaben,DE276,"Augsburg, Landkreis",NaN
23,Baden-Württemberg,Karlsruhe,DE121,"Baden-Baden, Stadtkreis",NaN
24,Bayern,Oberfranken,DE241,"Bamberg, Kreisfreie Stadt",NaN
25,Bayern,Oberfranken,DE245,"Bamberg, Landkreis",NaN


### I have fill all the NaNs with the correct name of Landkreis:

In [62]:
df['UZ'][8]= str('AS')
df['UZ'][12]= str('AN')
df['UZ'][13]= str('AN')
df['UZ'][14]= str('AB')
df['UZ'][15]= str('AB')
df['UZ'][16]= str('A')
df['UZ'][17]= str('A')
df['UZ'][23]= str('BAD')
df['UZ'][24]= str('BA')
df['UZ'][25]= str('BA')
df['UZ'][28]= str('BT')
df['UZ'][29]= str('BT')
df['UZ'][32]= str('B')
df['UZ'][35]= str('BI')
df['UZ'][37]= str('BO')
df['UZ'][39]= str('BA')
df['UZ'][41]= str('BOT')
df['UZ'][42]= str('BRB')
df['UZ'][43]= str('BS')
df['UZ'][45]= str('HB')
df['UZ'][46]= str('HB')
df['UZ'][53]= str('C')
df['UZ'][55]= str('CO')
df['UZ'][56]= str('CO')
df['UZ'][59]= str('BRB')
df['UZ'][63]= str('DA')
df['UZ'][66]= str('DEL')
df['UZ'][67]= str('DE')
df['UZ'][69]= str('DLG')
df['UZ'][74]= str('DO')
df['UZ'][75]= str('DD')
df['UZ'][76]= str('DU')
df['UZ'][78]= str('D')
df['UZ'][83]= str('EA')
df['UZ'][85]= str('EMD')
df['UZ'][91]= str('EF')
df['UZ'][92]= str('ER')
df['UZ'][95]= str('E')
df['UZ'][99]= str('FL')
df['UZ'][101]= str('FT')
df['UZ'][102]= str('FF')
df['UZ'][103]= str('F')
df['UZ'][104]= str('FR')
df['UZ'][108]= str('FRI')
df['UZ'][111]= str('FÜ')
df['UZ'][114]= str('GE')
df['UZ'][115]= str('G')
df['UZ'][129]= str('HA')
df['UZ'][130]= str('HAL')
df['UZ'][112]= str('FÜ')
df['UZ'][131]= str('HH')
df['UZ'][133]= str('HAM')
df['UZ'][139]= str('HD')
df['UZ'][141]= str('HN')
df['UZ'][142]= str('HN')
df['UZ'][146]= str('HER')
df['UZ'][153]= str('HO')
df['UZ'][154]= str('HO')
df['UZ'][159]= str('IN')
df['UZ'][160]= str('J')
df['UZ'][162]= str('KL')
df['UZ'][163]= str('KL')
df['UZ'][164]= str('KA')
df['UZ'][165]= str('KA')
df['UZ'][166]= str('KS')
df['UZ'][167]= str('KS')
df['UZ'][168]= str('KF')
df['UZ'][170]= str('KE')
df['UZ'][171]= str('KI')
df['UZ'][174]= str('KO')
df['UZ'][176]= str('KR')
df['UZ'][181]= str('K')
df['UZ'][183]= str('LD')
df['UZ'][185]= str('LA')
df['UZ'][186]= str('LA')
df['UZ'][189]= str('L')
df['UZ'][190]= str('LEV')
df['UZ'][196]= str('LU')
df['UZ'][199]= str('HL')
df['UZ'][202]= str('MD')
df['UZ'][207]= str('MZ')
df['UZ'][209]= str('MA')
df['UZ'][215]= str('MM')
df['UZ'][224]= str('MG')
df['UZ'][225]= str('MÜ')
df['UZ'][226]= str('MH')
df['UZ'][227]= str('M')
df['UZ'][228]= str('M')
df['UZ'][229]= str('MS')
df['UZ'][233]= str('NM')
df['UZ'][234]= str('NMS')
df['UZ'][236]= str('NEA')
df['UZ'][237]= str('NEW')
df['UZ'][238]= str('NW')
df['UZ'][246]= str('N')
df['UZ'][250]= str('OB')
df['UZ'][255]= str('OF')
df['UZ'][256]= str('OF')
df['UZ'][257]= str('OL')
df['UZ'][258]= str('OL')
df['UZ'][261]= str('OS')
df['UZ'][262]= str('OS')
df['UZ'][269]= str('PA')
df['UZ'][270]= str('PA')
df['UZ'][272]= str('PAF')
df['UZ'][273]= str('PF')
df['UZ'][275]= str('PS')
df['UZ'][277]= str('PM')
df['UZ'][284]= str('R')
df['UZ'][285]= str('R')
df['UZ'][286]= str('H')
df['UZ'][287]= str('SB')
df['UZ'][289]= str('RS')
df['UZ'][302]= str('RO')
df['UZ'][303]= str('RO')
df['UZ'][304]= str('HRO')
df['UZ'][305]= str('HRO')
df['UZ'][315]= str('SZ')
df['UZ'][321]= str('SC')
df['UZ'][325]= str('SW')
df['UZ'][326]= str('SW')
df['UZ'][327]= str('SN')
df['UZ'][333]= str('SG')
df['UZ'][335]= str('SP')
df['UZ'][344]= str('SR')
df['UZ'][346]= str('S')
df['UZ'][316]= str('SZ')
df['UZ'][347]= str('AC')
df['UZ'][348]= str('SHL')
df['UZ'][356]= str('TR')
df['UZ'][362]= str('UL')
df['UZ'][378]= str('WEN')
df['UZ'][380]= str('WE')
df['UZ'][388]= str('WI')
df['UZ'][389]= str('WHV')
df['UZ'][393]= str('WOB')
df['UZ'][394]= str('WO')
df['UZ'][395]= str('WUN')
df['UZ'][396]= str('W')
df['UZ'][397]= str('WÜ')
df['UZ'][398]= str('WÜ')
df['UZ'][400]= str('Z')


In [63]:
df_nan=df[df['UZ'].isnull()]
df_nan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 98 to 98
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   NUTS 1.1  1 non-null      object
 1   NUTS 2.1  1 non-null      object
 2   Code      1 non-null      object
 3   Name      1 non-null      object
 4   UZ        0 non-null      object
dtypes: object(5)
memory usage: 48.0+ bytes


In [64]:
df_nan.head(12)

,NUTS 1.1,NUTS 2.1,Code,Name,UZ
98,EXTRA-REGIO NUTS 1.mw-parser-output .fussnoten...,Extra-Regio NUTS 2 *,DEZZZ,Extra-Regio NUTS 3 *,NaN


In [65]:
df.head(3)

,NUTS 1.1,NUTS 2.1,Code,Name,UZ
0,Rheinland-Pfalz,Koblenz,DEB12,Ahrweiler,AW
1,Bayern,Schwaben,DE275,Aichach-Friedberg,"AIC, FDB"
2,Baden-Württemberg,Tübingen,DE145,Alb-Donau-Kreis,UL


In [66]:
df['UZ'] = df['UZ'].str.split(',', n=1).str.get(0)





In [67]:
df['UZ'].unique()

array(['AW', 'AIC', 'UL', 'ABG', 'AK', 'SAW', 'AÖ', 'AZ', 'AS', 'WST',
       'ABI', 'AN', 'AB', 'A', 'AUR', 'DÜW', 'KG', 'KH', 'TÖL', 'BAD',
       'BA', 'BAR', 'BZ', 'BT', 'BGL', 'HP', 'B', 'WIL', 'BC', 'BI',
       'BIR', 'BO', 'FN', 'BOR', 'BOT', 'BRB', 'BS', 'FR', 'HB', 'BLK',
       'BB', 'BK', 'CW', 'CE', 'CHA', 'C', 'CLP', 'CO', 'COC', 'COE',
       'CUX', 'DAH', 'LDS', 'DA', 'DEG', 'DEL', 'DE', 'DH', 'DLG', 'DGF',
       'HEI', 'DON', 'KIB', 'DO', 'DD', 'DU', 'DN', 'D', 'EBE', 'EIC',
       'EI', 'BIT', 'EA', 'EE', 'EMD', 'EM', 'EL', 'EN', 'PF', 'ED', 'EF',
       'ER', 'ERH', 'ERZ', 'E', 'ES', 'EU', nan, 'FL', 'FO', 'FT', 'FF',
       'F', 'FS', 'FDS', 'FRG', 'FRI', 'FD', 'FFB', 'FÜ', 'GAP', 'GE',
       'G', 'GER', 'GI', 'GF', 'GS', 'GTH', 'NOH', 'GRZ', 'GG', 'GP',
       'GR', 'GÖ', 'GZ', 'GT', 'HA', 'HAL', 'HH', 'HM', 'HAM', 'WL', 'HZ',
       'HVL', 'HAS', 'HK', 'HD', 'HDH', 'HN', 'HS', 'HE', 'HF', 'HER',
       'HEF', 'RZ', 'HBN', 'HI', 'HSK', 'HG', 'HO', 'KÜN', 'HOL', '

In [68]:
df.head(20)

,NUTS 1.1,NUTS 2.1,Code,Name,UZ
0,Rheinland-Pfalz,Koblenz,DEB12,Ahrweiler,AW
1,Bayern,Schwaben,DE275,Aichach-Friedberg,AIC
2,Baden-Württemberg,Tübingen,DE145,Alb-Donau-Kreis,UL
3,Thüringen,Thüringen,DEG0M,Altenburger Land,ABG
4,Rheinland-Pfalz,Koblenz,DEB13,Altenkirchen (Westerwald),AK
5,Sachsen-Anhalt,Sachsen-Anhalt,DEE04,Altmarkkreis Salzwedel,SAW
6,Bayern,Oberbayern,DE214,Altötting,AÖ
7,Rheinland-Pfalz,Rheinhessen-Pfalz,DEB3B,Alzey-Worms,AZ
8,Bayern,Oberpfalz,DE231,"Amberg, Kreisfreie Stadt",AS
9,Bayern,Oberpfalz,DE234,Amberg-Sulzbach,AS


In [69]:
df.rename(columns={'UZ':'Landkreis'}, inplace=True)
df.set_index('Code', inplace=True)


df.to_csv('Landkreis_ID.csv')

In [70]:
df.head()

,NUTS 1.1,NUTS 2.1,Name,Landkreis
Code,,,,
DEB12,Rheinland-Pfalz,Koblenz,Ahrweiler,AW
DE275,Bayern,Schwaben,Aichach-Friedberg,AIC
DE145,Baden-Württemberg,Tübingen,Alb-Donau-Kreis,UL
DEG0M,Thüringen,Thüringen,Altenburger Land,ABG
DEB13,Rheinland-Pfalz,Koblenz,Altenkirchen (Westerwald),AK
